# Lag Time Analysis: Prepare reach data

In [1]:
import hydrofunctions as hf
print("Hydrofunctions version: ", hf.__version__)

import pandas as pd
pd.set_option('display.max_columns', None) # This changes the default to show all 35 columns of our data when we display it.

import matplotlib
#%matplotlib notebook
# Use the following for JupyterLab instead:
%matplotlib inline

import analysis_functions
print("Analysis_functions version: ", analysis_functions.__version__)

Hydrofunctions version:  0.2.4
Analysis_functions version:  2025.05.19


# Reach Setup
Create a table of reaches for use in the analysis.

In [2]:
stations = pd.read_parquet('stations.parquet')
stations

,name,position_nhd_m,gauge_datum_m,basin_area_km2,medianQ_cms,color,StageQ0root,StageMin
01541200,Curwensville,507322,342.796368,951.0,9.967514,#4a58dd,0.579685,0.701040
01541303,Hyde,494288,333.420720,1228.0,14.186717,#27d7c4,0.645428,0.835152
01542500,Karthaus,422588,253.163832,3787.0,40.209856,#95fb51,-0.016348,0.280416
01545500,Renovo,367064,193.301112,7705.0,84.667232,#ffa423,-0.474518,-0.115824
01551500,Williamsport,273292,150.869904,14716.0,161.972096,#ba2208,-0.581263,-0.100584


In [3]:
# Create a table of reach information.
# Define reaches as pairs of UpStream and DownStream stations.
reach_order = [(0,1), (1,2), (2,3), (3,4)]
reach_index = []
reach_data = []
# Create a set of standard symbols and colors for each reach
symbol = iter(['+',  'x', '*', 'o'])
color = iter(['#2f9df5', '#4df884', '#dedd32', '#f65f18'])
for reach in reach_order:
    us, ds = reach
    name = f"{stations.iloc[us]['name']}-{stations.iloc[ds]['name']}"
    reach_index.append(name)
    
    length = stations.iloc[us]['position_nhd_m']-stations.iloc[ds]['position_nhd_m'] # Length of the reach in meters
    us_height = stations.iloc[us].gauge_datum_m + stations.iloc[us].StageQ0root # Elevation in meters above the NGVD29 of the estimated stage for zero discharge stage
    ds_height = stations.iloc[ds].gauge_datum_m + stations.iloc[ds].StageQ0root
    head = us_height - ds_height # drop in elevation from US to DS station; meters.
    ds_Q_growth = stations.iloc[ds].medianQ_cms / stations.iloc[us].medianQ_cms # Increase in median Q relative to US median Q
    ds_A_growth = stations.iloc[ds].basin_area_km2 / stations.iloc[us].basin_area_km2 # Increase in basin Area relative to US basin Area

    reach_data.append({
        'us'     : stations.iloc[us].name,
        'ds'     : stations.iloc[ds].name,
        'length' : length,
        'head'   : head,
        'gradient': head / length,
        'deltaQ' : ds_Q_growth,
        'deltaA' : ds_A_growth,
        'symbol': next(symbol),
        'color': next(color)
    })
reaches = pd.DataFrame(data=reach_data, index=reach_index, columns=['us', 'ds', 'length', 'head', 'gradient', 'deltaQ', 'deltaA', 'symbol', 'color'])
reaches.to_parquet('reaches.parquet', compression='gzip')
reaches

,us,ds,length,head,gradient,deltaQ,deltaA,symbol,color
Curwensville-Hyde,01541200,01541303,13034,9.309905,0.000714,1.423295,1.291272,+,#2f9df5
Hyde-Karthaus,01541303,01542500,71700,80.918664,0.001129,2.834331,3.083876,x,#4df884
Karthaus-Renovo,01542500,01545500,55524,60.320890,0.001086,2.105634,2.034592,*,#dedd32
Renovo-Williamsport,01545500,01551500,93772,42.537953,0.000454,1.913043,1.909929,o,#f65f18
